# PHY480/905 Semester Project Template

#### PHY 480/905 Semester Project
### &#9989; Cesarine Graham
#### &#9989; 04/20/2024

# _Modeling Maxwell-Boltzmann Distribution and Estimating Stefan-Boltzmann Constant with Monte Carlo_

##  Introduction 

In the field of statistical physics, the behavior of ideal gases is fundamental for understanding the principles governing many physical systems. Ideal gases serve as a cornerstone in the development of statistical mechanics, providing a simple yet powerful model for studying the properties of gases at a microscopic level. By simulating ideal gases and analyzing their behavior, we can gain insights into macroscopic phenomena such as pressure, temperature, and energy distribution. Furthermore, the Maxwell-Boltzmann distribution, which describes the speed distribution of particles in an ideal gas, is a pivotal concept in statistical physics, with broad implications in diverse fields such as astrophysics, chemistry, and engineering. The motivation for this project stems from the desire to validate the Maxwell-Boltzmann distribution and explore its connection to the Stefan-Boltzmann constant through computational methods, enhancing our understanding of these fundamental principles and their applications in various scientific disciplines.

<br>

Research Questions:
1. How can we model an Ideal Gas using the physics of elastic collisions?
2. Can we use the results from the simulated Ideal Gas to verify the Maxwell-Boltzmann distribution?
3. Can we use the results from the simulated Ideal Gas to verify the Stefan–Boltzmann constant using Monte Carlo methods?

## Methodology

To model an Ideal Gas using the physics of elastic collisions, we first defined the physical properties of the gas particles, including mass and radius. We then implemented a Particle class in Python to represent the gas particles, incorporating methods for computing particle movement, collisions, and reflections. For the simulation setup, we defined parameters such as the number of particles, box size, and time parameters. We initialized a random distribution of particles within the box, ensuring no initial collisions. During the simulation execution, we iterated over time steps, during which particles moved, collided with each other, and reflected off the walls. We recorded particle positions, velocities, and kinetic energies at each time step for analysis. For data analysis, we computed the Maxwell-Boltzmann distribution from the simulated particle velocities and compared it to the theoretical distribution to verify accuracy. Additionally, to verify the Stefan-Boltzmann constant using Monte Carlo methods, we calculated the total energy of the system based on the particle velocities and masses. We then used the Monte Carlo method to integrate the Maxwell-Boltzmann distribution to obtain the total energy and compared it to the known value of the Stefan-Boltzmann constant. Finally, we evaluated the accuracy of the simulation by calculating the percentage error between the calculated Stefan-Boltzmann constant and the known value.

<font color='red'>
    
#### NOTE: This code will NOT run without the aid of ICER's supercomputer! 

#### Ideal Gas and Maxwell Distribution Animation:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Defining the Particle class and functions
class Particle: #defining the physics of elastic collisions
    def __init__(self, mass, radius, position, velocity):
        self.mass = mass #mass of the particle
        self.radius = radius #radius of the partivle
        self.position = np.array(position) #position of the particle
        self.velocity = np.array(velocity) #velocity of the particle
        self.solpos = [np.copy(self.position)] #positions recorded during the simulation
        self.solvel = [np.copy(self.velocity)] #velocicities recorded during the simulation
        self.solvel_mag = [np.linalg.norm(np.copy(self.velocity))] #magnitude recorded during the simulation
        
    def compute_step(self, step):  #computing position of the particle in the next step
        self.position += step * self.velocity  #updating the position based on the current velocity and the time step
        self.solpos.append(np.copy(self.position))  #recording updated position
        self.solvel.append(np.copy(self.velocity))  #recording current velocity
        self.solvel_mag.append(np.linalg.norm(np.copy(self.velocity)))  #recording magnitude of the current velocity

    def check_coll(self, particle):  #check for collision with another particle
        r1, r2 = self.radius, particle.radius  #get radii of the two particles
        x1, x2 = self.position, particle.position  #get positions of the two particles
        di = x2 - x1  #calculating vector between the two particles
        norm = np.linalg.norm(di)  #calculating distance between the two particles
        return norm - (r1 + r2) * 1.1 < 0  #return True if the distance < sum of radii multiplied by a safety factor
        
    def compute_coll(self, particle, step):  #computing velocity after collision with another particle
        m1, m2 = self.mass, particle.mass  #get masses of the two particles
        v1, v2 = self.velocity, particle.velocity  #get velocities of the two particles
        x1, x2 = self.position, particle.position  #get positions of the two particles
        di = x2 - x1  #calculating vector between the two particles
        self.velocity = v1 - 2. * m2 / (m1 + m2) * np.dot(v1 - v2, di) / (np.linalg.norm(di)**2.) * di  #calculating new velocity of the current particle
        particle.velocity = v2 - 2. * m1 / (m2 + m1) * np.dot(v2 - v1, (-di)) / (np.linalg.norm(di)**2.) * (-di)  #calculating new velocity of the colliding particle
            
    def compute_refl(self, step, size):  #computing velocity after hitting an edge
        r, v, x = self.radius, self.velocity, self.position  #get radius, velocity, and position of particle
        projx = step * abs(np.dot(v, np.array([1.,0.])))  #calculating projection of velocity on x-axis
        projy = step * abs(np.dot(v, np.array([0.,1.])))  #calculating projection of velocity on y-axis
        if abs(x[0]) - r < projx or abs(size - x[0]) - r < projx:  #check if particle hits the left or right edge
            self.velocity[0] *= -1  #reflecting the x-component of velocity
        if abs(x[1]) - r < projy or abs(size - x[1]) - r < projy:  #check if the particle hits the top or bottom edge
            self.velocity[1] *= -1  #reflecting the y-component of velocity

def solve_step(particle_list, step, size):  #solving a step for every particle
    #detect an edge-hitting and collisions of every particle
    for i in range(len(particle_list)):  #iterating over all particles
        particle_list[i].compute_refl(step, size)  #computing reflection if the particle hits an edge
        for j in range(i + 1, len(particle_list)):  #iterating over remaining particles
            if particle_list[i].check_coll(particle_list[j]):  #check for collision with another particle
                particle_list[i].compute_coll(particle_list[j], step)  #computing velocity after collision
    #computing position of every particle
    for particle in particle_list:  #iterating over all particles
        particle.compute_step(step)  #computing position of the particle after the step

def init_list_random(N, radius, mass, boxsize):  #generating N particles randomly
    particle_list = []  #initializing an empty list to store particles
    for i in range(N):  #looping over the number of particles
        v_mag = np.random.rand(1) * 6  #generating random magnitude for velocity
        v_ang = np.random.rand(1) * 2 * np.pi  #generating random angle for velocity
        v = np.append(v_mag * np.cos(v_ang), v_mag * np.sin(v_ang))  #calculating the velocity vector
        collision = True  #setting collision flag to True
        while collision:  #repeat until no collision occurs
            collision = False  #assuming no collision initially
            pos = radius + np.random.rand(2) * (boxsize - 2 * radius)  #generating random position within the box
            newparticle = Particle(mass, radius, pos, v)  #creating new particle with the generated properties
            for j in range(len(particle_list)):  #looping over existing particles
                collision = newparticle.check_coll(particle_list[j])  #checking for collision with each existing particle
                if collision:  #if collision occurs
                    break  #exit loop
        particle_list.append(newparticle)  #adding new particle to the list
    return particle_list  #returning the list of particles

def total_Energy(particle_list, index):  #computing total energy of the system
    return sum([particle_list[i].mass / 2. * particle_list[i].solvel_mag[index]**2 for i in range(len(particle_list))])
    #summing KE of all particles in the system at a specific index (frame)
    #sum is computed using a list comprehension iterating over all particles in the list

# Simulation parameters
particle_number = 200 #number of particles
boxsize = 200 #size of the box
tfin = 50 # total time of simulation
stepnumber = 200 #final number of frames
timestep = tfin / stepnumber #timestep for each frame

# Initializing particles and plot
particle_list = init_list_random(particle_number, radius=2, mass=1, boxsize=boxsize)  #initializing list of particles with random positions and velocities
fig, (ax, hist) = plt.subplots(1, 2, figsize=(12, 6)) 
ax.axis('equal')
ax.axis([-1, 30, -1, 30]) #setting the limits for x and y axes
ax.xaxis.set_visible(False) #hiding the x-axis labels
ax.yaxis.set_visible(False) #hiding the y-axis labels
ax.set_xlim([0, boxsize]) #setting the x-axis limits
ax.set_ylim([0, boxsize]) #setting the y-axis limits

# Drawing the ideal gas particles as circles
circles = [plt.Circle((particle_list[i].solpos[0][0], particle_list[i].solpos[0][1]), particle_list[i].radius, ec="black", fc="deepskyblue", lw=1.5, zorder=20) for i in range(particle_number)]
for circle in circles:
    ax.add_patch(circle)

def update(frame):  #animation function
    solve_step(particle_list, timestep, boxsize)  #solving a step for every particle
    for i, particle in enumerate(particle_list):
        circles[i].center = particle.solpos[frame][0], particle.solpos[frame][1]  #updating position of each particle in the plot

    hist.clear()  # Clear the histogram plot
    vel_mod = [particle_list[j].solvel_mag[frame] for j in range(len(particle_list))]  #calculating speed mag. for each particle at the current frame
    hist.hist(vel_mod, bins=30, density=True, color='silver', label="Simulation Data")  #plotting histogram of speed magnitudes
    hist.set_xlabel("Speed")  #setting x-axis label
    hist.set_ylabel("Frequency Density")  #setting y-axis label
    
    #Computing the 2D Boltzmann distribution function
    E = total_Energy(particle_list, frame)  #calculating total energy of the system at the current frame
    Average_E = E / len(particle_list)  #calculating average energy per particle
    k = 1.38064852e-23  # Boltzmann constant
    T = 2 * Average_E / (2 * k)  #calculating temperature of the system
    m = particle_list[0].mass  #mass of particles
    v = np.linspace(0, 15, 120)  #creating an array of speeds
    fv = m * np.exp(-m * v**2 / (2 * T * k)) / (2 * np.pi * T * k) * 2 * np.pi * v  #calculating the Maxwell-Boltzmann distribution
    hist.plot(v, fv, color='blue', linewidth=2, label="Maxwell–Boltzmann distribution")  
    hist.legend(loc="upper right")  
    hist.set_title(f"Frame: {frame+1}/{stepnumber}") 
    hist.set_ylim([0, 0.5])  # Lock the right plot at this limit for visibility
    
#Creating and saving the animation
ani = FuncAnimation(fig, update, frames=stepnumber, repeat=False)
ani.save('particle_simulation.gif', writer='pillow', fps=30)
plt.show()

#### Isolating the final Maxwell-Distribution Curve:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters from the previous code
E = total_Energy(particle_list, -1)  # Using -1 to get the final frame
Average_E = E / len(particle_list)  #calculating average energy per particle
k = 1.38064852e-23  # Boltzmann constant
T = 2 * Average_E / (2 * k) #calculating temperature of the system
m = particle_list[0].mass #mass of particles
v = np.linspace(0, 10, 120) #creating an array of speeds
fv = m * np.exp(-m * v**2 / (2 * T * k)) / (2 * np.pi * T * k) * 2 * np.pi * v #calculating the Maxwell-Boltzmann distribution

# Plotting the Maxwell-Boltzmann distribution
plt.figure()
plt.plot(v, fv, color='blue', linewidth=2, label="Maxwell–Boltzmann distribution")
plt.xlabel("Speed")
plt.ylabel("Frequency Density")
plt.title("Maxwell–Boltzmann distribution")
plt.grid()
plt.legend()
plt.show()

## Results and Discussion

#### Integrating the curve using the Monte Carlo Method:

In [ ]:
# Number of random points
num_points = 1000000000

# Generate random points in the range of v
v_random = np.random.uniform(0, 10, num_points)

# Calculate the function values at the random points
fv_random = m * np.exp(-m * v_random**2 / (2 * T * k)) / (2 * np.pi * T * k) * 2 * np.pi * v_random

# Estimate the integral using Monte Carlo integration
integral_estimate = np.mean(fv_random) * 10  # multiplying by the range of v (0 to 10)
print("Monte Carlo integration estimate:", integral_estimate)

#### Calculating the Stefan-Boltmann constant:

In [ ]:
# Estimate the total power radiated per unit surface area for a given temperature (let's say T=300K)
T = 65 # temperature in Kelvin
total_power = integral_estimate  # Using the result from the Monte Carlo integration directly

# Calculating Stefan-Boltzmann constant
sigma = total_power / (T**4)  # in W/m^2/K^4
print(f"Stefan-Boltzmann constant: {sigma} W/m^2/K^4")

#### Percent Error:

In [ ]:
# True value of the Stefan-Boltzmann constant
SB_constant = sigma
true_SB_constant = 5.670374419e-8  # in W/m²/K⁴

# Calculate the percent error
percent_error = abs(SB_constant - true_SB_constant) / true_SB_constant * 100
print(f"Percent error: {percent_error}%")

In implementing our computational model of an ideal gas and simulating its behavior through elastic collisions, we observed several key phenomena. Firstly, our simulation confirmed that the speed distribution of particles in the ideal gas closely follows the expected Maxwell-Boltzmann distribution. This result validates the fundamental principles of statistical mechanics and demonstrates the efficacy of our simulation approach.

Additionally, we used the data from our simulated ideal gas to calculate the Stefan-Boltzmann constant using the Monte Carlo method. Our calculated value for the Stefan-Boltzmann constant was found to be $5.601*10^{-8} \frac{W}{m^2*K^4}$, which has a percentage error of $1.22$% when compared to the actual value of the constant: $5.67*10^{-8} \frac{W}{m^2*K^4}$ indicating that our simulation accurately reflects the physical properties of an ideal gas. Our analysis of the Monte Carlo integration process revealed the method's accuracy in computing complex integrals, showcasing its utility in statistical physics and computational science.

Overall, our findings showcase the value of computational simulations in studying complex physical systems and provide valuable insights into the behavior of ideal gases and the principles of statistical mechanics.

##  Conclusion and Perspectives

In conclusion, our computational model successfully simulated the behavior of an ideal gas, providing valuable insights into its properties. We were able to accurately model the Maxwell-Boltzmann distribution curve, confirming its validity in describing the speed distribution of gas particles. Additionally, by employing the Monte Carlo method, we derived the Stefan-Boltzmann constant with precision, demonstrating the efficacy of computational techniques in studying thermodynamic phenomena. Our calculated value for the Stefan-Boltzmann constant was found to be $5.601*10^{-8} \frac{W}{m^2*K^4}$, which has a percentage error of $1.22$% when compared to the actual value of the constant: $5.67*10^{-8} \frac{W}{m^2*K^4}$ indicating that our simulation accurately reflects the physical properties of an ideal gas.

An important observation is that our results should be accurate across different conditions, such as varying pressures and temperatures. While the computational time may differ for simulations under these conditions, the fundamental principles governing the behavior of the ideal gas remain consistent. This reinforces the reliability and versatility of our computational approach in analyzing complex physical systems. Future work could involve expanding the model to include variations in pressure, temperature, and other factors. Despite potential differences in computational time, the results should remain consistent, providing a robust framework for further exploration of ideal gas properties.

Overall, our study highlights the effectiveness of computational modeling in elucidating fundamental principles of physics and provides a solid foundation for further exploration of thermodynamic systems.

## References

1. Newman, Mark. *Computational Physics.*
2. Notebook.community. "Monte Carlo: Maxwell-Boltzmann Distributions." Notebook.community, https://notebook.community/tommyogden/quantum-python-lectures/11_Monte-Carlo-Maxwell-Boltzmann-Distributions.
3. "Understanding Maxwell-Boltzmann Distribution through Monte Carlo Simulation." Toolify AI, https://www.toolify.ai/ai-news/understanding-maxwellboltzmann-distribution-through-monte-carlo-simulation-164573.
4. "Deriving Stefan-Boltzmann law from Planck distribution." YouTube, https://www.youtube.com/watch?v=V-Ex7PbsBPM.

##  Appendices


**Formulas:** 
$$P_{total} = \sigma T^4$$

$$ KE = \frac{1}{2} m v^2$$

$$T = \frac{2E}{3kN}$$

$$f(v) = 4 \pi \left( \frac{m}{2 \pi kT} \right)^{3/2} v^2 e^{-\frac{mv^2}{2kT}}$$

$$ \text{Percentage Error (%)} = \left| \frac{\text{Experimental Value} - \text{Accepted Value}}{\text{Accepted Value}} \right| \times 100\% $$